In [1]:
import pandas as pd

# 1) 이미지 쪽 요약 파일 (음식당 1행짜리)
one_df = pd.read_csv("oneimage_food_label.csv")   # 네가 방금 만든 파일

# 2) True/False 정보가 들어있는 식약처 확장 CSV
true_df = pd.read_csv("food_ingredients_True.csv")

# 3) 우리가 관심 있는 6가지 컬럼만 뽑기
flag_cols = ["대두_땅콩_여부", "유제품여부", "탄닌가능성", "쿠마린여부", "카페인여부", "알코올_유무"]

true_small = true_df[["식품명"] + flag_cols].copy()

# 4) 이미지-매칭 음식(oneimage)의 식품명(B열)과 True 테이블 join
merged = pd.merge(
    one_df,
    true_small,
    left_on="식품명(B열)",   # oneimage 쪽
    right_on="식품명",       # True 테이블 쪽
    how="left",
)

# NaN은 False로 간주 (안 적혀 있으면 성분 없음 취급)
merged[flag_cols] = merged[flag_cols].fillna(False).astype(bool)

print("음식 개수:", len(merged))
merged.head()


음식 개수: 830


C:\Users\EL19\AppData\Local\Temp\ipykernel_13208\3043594328.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged[flag_cols] = merged[flag_cols].fillna(False).astype(bool)


,json_path,image_file,food_from_filename,식품코드,식품명(B열),match_type,n_images_for_food,식품명,대두_땅콩_여부,유제품여부,탄닌가능성,쿠마린여부,카페인여부,알코올_유무
0,TL1/B/11/B11001/03/측면/B_11_B11001_BLT샌드위치_03_0...,B_11_B11001_BLT샌드위치_03_08.jpg,BLT샌드위치,NaN,NaN,not_found,230,NaN,False,False,False,False,False,False
1,TL1/A/14/A14001/13/정위/A_14_A14001_가라아케_13_09.json,A_14_A14001_가라아케_13_09.jpg,가라아케,NaN,NaN,not_found,310,NaN,False,False,False,False,False,False
2,TL10/B/12/B12001/10/측면/B_12_B12001_가리비초밥_10_07...,B_12_B12001_가리비초밥_10_07.jpg,가리비초밥,NaN,NaN,not_found,220,NaN,False,False,False,False,False,False
3,TL1/A/14/A14002/13/정위/A_14_A14002_가이 팟 퐁 커리_13...,A_14_A14002_가이 팟 퐁 커리_13_09.jpg,가이 팟 퐁 커리,NaN,NaN,not_found,360,NaN,False,False,False,False,False,False
4,TL1/A/13/A13001/30/정위/A_13_A13001_가자미구이_30_09....,A_13_A13001_가자미구이_30_09.jpg,가자미구이,NaN,NaN,not_found,280,NaN,False,False,False,False,False,False


In [2]:
# 6개 카테고리 중 하나라도 True인 음식만 필터
mask_any = merged[flag_cols].any(axis=1)
affected = merged[mask_any]

print("True가 하나 이상인 음식 개수:", len(affected))

# 카테고리별로 True 개수 세기
counts_per_flag = affected[flag_cols].sum().sort_values(ascending=False)
print("\n카테고리별 True 개수:")
print(counts_per_flag)


True가 하나 이상인 음식 개수: 140

카테고리별 True 개수:
유제품여부       126
카페인여부        19
탄닌가능성        14
대두_땅콩_여부     10
쿠마린여부         4
알코올_유무        2
dtype: int64
